<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/classificacao_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils

In [12]:
(previsores_treinamento, classe_treinamento), (previsores_teste, classe_teste) = mnist.load_data()
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_teste = previsores_teste.astype('float32') / 255

In [13]:
previsores_treinamento.shape, classe_treinamento.shape, previsores_teste.shape, classe_teste.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [15]:
previsores_treinamento = previsores_treinamento.reshape((len(previsores_treinamento), 
                                                         np.prod(previsores_treinamento.shape[1:])))

previsores_teste = previsores_teste.reshape((len(previsores_teste), np.prod(previsores_teste.shape[1:])))

In [16]:
previsores_treinamento.shape, previsores_teste.shape

((60000, 784), (10000, 784))

In [21]:
classe_dummy_treinamento = np_utils.to_categorical(classe_treinamento)
classe_dummy_teste = np_utils.to_categorical(classe_teste)

# Estrutura da rede AUTOENCODER para redução de dimensionalidade
Camada de entrada 784

Camada oculta 32

Camada de saída 784

In [17]:
auto_encoder = Sequential()

auto_encoder.add(Dense(units = 32, activation = 'relu', input_dim = 784))
auto_encoder.add(Dense(units = 784, activation = 'sigmoid'))
auto_encoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_3 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [ ]:
auto_encoder.compile(optimizer = 'sgd', loss = 'binary_crossentropy',
                     metrics = ['accuracy'])

auto_encoder.fit(previsores_treinamento, previsores_treinamento,
                 epochs = 100, batch_size = 256,
                 validation_data = (previsores_teste, previsores_teste));

In [10]:
dimensao_original = Input(shape=(784,))
camada_encoder = auto_encoder.layers[0]
encoder = Model(dimensao_original, camada_encoder(dimensao_original))

In [18]:
previsores_treinamento_codificado = encoder.predict(previsores_treinamento)
previsores_treinamento_codificado.shape

(60000, 32)

In [19]:
previsores_teste_codificado = encoder.predict(previsores_teste)
previsores_teste_codificado.shape

(10000, 32)

# Criação de uma rede densa para previsores com e sem redução de dimensionalidade.

Sem redução de dimensionalidade

In [ ]:
model_sem_reducao = Sequential()

model_sem_reducao.add(Dense(units = 400, activation = 'relu', input_dim = 784))
model_sem_reducao.add(Dense(units = 400, activation = 'relu'))
model_sem_reducao.add(Dense(units = 10, activation = 'softmax'))

model_sem_reducao.compile(optimizer = 'sgd', 
                          loss = 'categorical_crossentropy',
                          metrics = ['accuracy'])

model_sem_reducao.fit(previsores_treinamento, classe_dummy_treinamento,
                      batch_size = 256, epochs = 100,
                      validation_data = (previsores_teste, classe_dummy_teste))

In [27]:
np.mean(model_sem_reducao.history.history['val_accuracy'])

0.9520769995450974

Com redução de dimensionalidade

In [ ]:
model_com_reducao = Sequential()

model_com_reducao.add(Dense(units = 21, activation = 'relu', input_dim = 32))
model_com_reducao.add(Dense(units = 21, activation = 'relu'))
model_com_reducao.add(Dense(units = 10, activation = 'softmax'))

model_com_reducao.compile(optimizer = 'sgd', 
                          loss = 'categorical_crossentropy',
                          metrics = ['accuracy'])

model_com_reducao.fit(previsores_treinamento_codificado, classe_dummy_treinamento,
                      batch_size = 256, epochs = 100,
                      validation_data = (previsores_teste_codificado, classe_dummy_teste))

In [28]:
np.mean(model_com_reducao.history.history['val_accuracy'])

0.7221419996023178

# Para uma inferência mais adequada, o ideal é aumentar o número de épocas consideralmente para o treinamento do modelo, e assim termos uma resposta melhor ao uso ou não de uma rede neural na redução de dimensionalidade dos dados.